In [14]:
# Initialize the random number generator
rng = MersenneTwister();
srand(2016);

In [15]:
# Size of the matrix
n = 64;

In [16]:
# Random initialization of matrix A
L = zeros(Float64,n,n)
U = zeros(Float64,n,n)
P = randperm(rng,n) # Randow row permutation
for i=1:n
    L[P[i],i] = 3 # Largest entry in the column
    L[P[i+1:n],i] = rand(rng, -2:2, n-i)
    U[i,i] = rand(rng, 1:2)
    U[i,i+1:n] = rand(rng, -2:2, n-i)
end
A = L * U
A0 = copy(A);
A0[1:6,1:6]

6x6 Array{Float64,2}:
  0.0   0.0   2.0   0.0  -5.0    3.0
 -2.0  -4.0   5.0   2.0  -1.0    1.0
  2.0   4.0  -5.0  -1.0  -2.0  -12.0
 -4.0  -4.0   4.0   2.0  -4.0   -1.0
  4.0   3.0   5.0  -6.0  -4.0    7.0
 -2.0  -1.0  -4.0   1.0   5.0   -5.0

In [17]:
# Initializing the right-hand side
xe = rand(rng, 0:9, n) # This will be our solution
b = A * xe
b'

1x64 Array{Float64,2}:
 -148.0  -52.0  -537.0  132.0  -154.0  …  -173.0  183.0  20.0  9.0  490.0

In [18]:
include("getrf.jl")

LoadError: LoadError: LoadError: UndefVarError: P_row not defined
while loading /Users/darve/git_directories/numerical_linear_algebra/Code/getrf.jl, in expression starting on line 121
while loading In[18], in expression starting on line 1

In [19]:
A = copy(A0)
P = getrf!(A)
# Solve
x = getrs(A, P, b)
@show norm(x-xe)
@assert x == xe # No roundoff error should occur for this example
println("getrf!: PASSED")

norm(x - xe) = 0.0
getrf!: PASSED


In [20]:
function flip_last_bit(A)
    B = Array{Float64}(size(A))
    for j=1:size(A,2)
        for i=1:size(A,1)
            last_bit_flpd = Int64( significand(A[i,j]) / eps(Float64) ) $ 1
            B[i,j] = (2.0^exponent(A[i,j])) * (Float64(last_bit_flpd) * eps(Float64))
        end
    end
    return B
end

flip_last_bit (generic function with 1 method)

In [21]:
# Test flip_last_bit()
ntrial = 128
x = (1<<20) * ( randn(ntrial,ntrial) - 0.5 )
# This test will fail if you modify flip_last_bit()
# You can still run the rest of the script safely
abs(x - flip_last_bit(x)) == eps(Float64) * (2.0.^exponent(x)) ? "TEST PASSED" : "TEST FAILED"

"TEST PASSED"

In [22]:
# Test the accuracy of the solver
# Random orthonormal matrix Q
X = rand(n,n)
Q,R = qr(X)
b = randn(n)
ntrial = 64
e = Array{Float64}(2,ntrial)
for k=1:ntrial
    A = Q * diagm([ ones(n-k); (1.0/2.0).^(1:k) ]) * Q.'
    B = flip_last_bit(A)    
    xe = A\b
    xf = B\b
    P = getrf!(A)
    x = getrs(A, P, b)
    e[1,k] = norm(x-xe) / norm(xe)
    e[2,k] = norm(xf - xe) / norm(xe)
end

In [23]:
using Plots
plotly()

Plots.PlotlyBackend()

In [24]:
# Re-run all cells to get the plot below
plot(e', lab=["Pivoted LU" "Perturbed Input"], yscale = :log, 
xlabel="Matrix index", ylabel="Error", left_margin = 50px)

In [25]:
generate_tex_pgfplot = false 
if generate_tex_pgfplot
    pgfplots()
    plt = plot(e', lab=["Pivoted LU" "Perturbed Input"], yscale = :log, 
xlabel="Matrix index", ylabel="Error", left_margin = 50px)
end

In [26]:
if generate_tex_pgfplot
    PGFPlots.save("error_pivoted_lu.tex", plt.o, include_preamble=false)
end